In [1]:
import os 
import sys
from datetime import date
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('utils'), '..')))

import utils

In [42]:
import requests
from bs4 import BeautifulSoup

movie_url= 'https://www.imdb.com/title/tt0198460/'

try:
    # Fetch the webpage content
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(movie_url, headers=headers)
    response.raise_for_status()
    
    # Parse the HTML
    soup = BeautifulSoup(response.text, 'lxml')
    # Extract title, year, rating, and genre
    title_element = soup.find('h1')
    title = title_element.text.strip() if title_element else None
    # Extract year
    year_element = soup.find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color', href=lambda href: href and '/releaseinfo' in href)
    year = int(year_element.text.strip()) if year_element else None
    rating_element = soup.find('span', class_="sc-bde20123-1 cMEQkK")
    rating = float(rating_element.text.strip()) if rating_element else None
    # Extract genres
    genre_elements = soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt', href=lambda href: href and '/search/title?genres=' in href)
    genres = [genre.find('span', class_='ipc-chip__text').text.strip() for genre in genre_elements]
    
    # # Extract runtime and certificate
    # Extract runtime
    # runtime_element = soup.find('li', class_='ipc-inline-list__item', string=lambda text: 'min' in text)
    # runtime = int(runtime_element.text.split(' ')[0]) if runtime_element else None
    # Extract certificate
    certificate_element = soup.find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color', href=lambda href: href and '/parentalguide/certificates' in href)
    certificate = certificate_element.text.strip() if certificate_element else None    
    # Extract director
    director_element = soup.find('div', class_='ipc-metadata-list-item__content-container', string=lambda text: 'Director' in text)
    director = director_element.find_next('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if director_element else None
    print(director)
    # Extract writers
    writers_element = soup.find('div', class_='ipc-metadata-list-item__content-container', string=lambda text: 'Writers' in text)
    writers = [writer.text.strip() for writer in writers_element.find_next('div').find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')] if writers_element else None

    # Extract stars
    stars_element = soup.find('div', class_='ipc-metadata-list-item__content-container', string=lambda text: 'Stars' in text)
    stars = [star.text.strip() for star in stars_element.find_next('div').find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')] if stars_element else None

    print(writers)
    # cast = ', '.join([actor.text.strip() for actor in details_section.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')]) if details_section else None
    
    # # Extract description
    # description_element = soup.find('div', class_='GenresAndPlot__TextContainerBreakpointXS_TO_M-cum89p-2')
    # description = description_element.text.strip() if description_element else None

    # movie_details = {
    #     'Title': title,
    #     'Year': year,
    #     'Rating': rating,
    #     'Genre': genre,
    #     'Runtime': runtime,
    #     'Director': director,
    #     'Cast': cast,
    #     'Description': description,
    #     'Certificate': certificate,
    #     'URL': movie_url
    # }
except (requests.RequestException, ValueError, AttributeError) as e:
    print(f"An error occurred while fetching details for {movie_url}: {e}")



TypeError: argument of type 'NoneType' is not iterable

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
movie_details = []
month_dict={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec' }

for year in range(2000, 2001):
    for month in range(1, 13):
        month_dir = f'../movie_links/{year}/{month:02d}'

        for day in range(1, 32):
            date_str = f'{year}-{month:02d}-{day:02d}'

            # Check if file for the date exists
            date_file = f'{month_dir}/{date_str}.txt'
            if not os.path.isfile(date_file):
                continue

            # Open the file and process links if it exists
            with open(date_file, 'r') as file:
                links = file.read().splitlines()
            for link in links:
                movie_details.append(utils.get_movie_details(link))

        print(f'Processed {month_dict[month]} {year}')


In [ ]:
movie_details[0]

In [ ]:
counter= [reading for reading in movie_details if reading!=None]

In [ ]:
df= pd.DataFrame(counter)

In [ ]:
df